### Exercício 3

Você deverá selecionar um banco de dados científico aberto entre as seguintes opções para realizar uma análise visual aprofundada, utilizando Python:

- PANGAEA (dados ambientais e climáticos)
- NASA Earth Data (observações de satélite)
- ENSEMBL / 1000 Genomes (dados genômicos)
- Global Biodiversity Information Facility (GBIF)
- UN Data (indicadores globais socioeconômicos)
- Figshare ou Zenodo (dados multidisciplinares de pesquisas científicas)

🎯 Tarefas:
1. Escolha um tema exclusivo dentro do banco de dados escolhido, que ainda não tenha sido selecionado por outro colega (registre seu tema antes de iniciar).
2. Baixe e faça a limpeza dos dados, garantindo organização para análise.
3. Utilize Python para produzir ao menos cinco visualizações diferentes, incluindo obrigatoriamente:
    - Gráfico de linha ou série temporal (quando aplicável)
    - Gráfico de dispersão ou correlação
    - Gráfico de distribuição (histograma, boxplot ou violin plot)
    - Visualização geoespacial (mapa estático ou interativo) — caso os dados tenham componente geográfico
    - Visualização avançada ou interativa à sua escolha (ex: gráfico de rede, heatmap, gráfico radial, dashboard simples)

4. Escreva um relatório científico (mínimo 500 palavras) contendo:
    - Contextualização do tema e relevância científica
    - Principais insights obtidos nas visualizações
    - Discussão crítica das limitações dos dados e da análise
    - Referências das bases de dados utilizadas


🛑 Regras Importantes:

- Cada aluno deve escolher um tema e banco de dados exclusivos.
- O código Python deve ser organizado, comentado e permitir reprodução da análise.
- As visualizações devem ser claras, esteticamente cuidadas e com legendas/títulos informativos.
- O relatório deve demonstrar domínio conceitual e capacidade de comunicação científica.

## Análise de Recursos Humanos em Pesquisa e Desenvolvimento: Uma Perspectiva Global dos Dados da ONU

**Dados Utilizados:**
- Fonte: United Nations Statistical Yearbook 2024
- URL: https://data.un.org/_Docs/SYB/CSV/SYB67_285_202411_Research%20and%20Development%20Expenditure%20and%20Staff.csv
- Período: 2005-2021
- Variáveis principais: Investimento em P&D (% PIB), Pesquisadores por milhão de habitantes

A importância estratégica dos recursos humanos em Pesquisa e Desenvolvimento (P&D) para o desenvolvimento econômico representa um dos pilares fundamentais da economia do conhecimento contemporânea.

O capital intelectual de uma nação, composto pelo capital interno (conhecimento armazenado em processos e sistemas), capital externo (conhecimento adquirido nas relações internacionais) e capital humano (as pessoas e suas competências), representa frequentemente um ativo que supera o patrimônio físico tradicional. Países como Coreia do Sul e Singapura exemplificam como investimentos maciços em educação e ciência e tecnologia podem resultar em crescimento exponencial do PIB, mesmo na ausência de abundantes recursos naturais.

A formação de recursos humanos altamente qualificados, especialmente em nível de doutorado, constitui um fator crucial para o avanço científico e a competitividade internacional. A Organização para a Cooperação e Desenvolvimento Econômico (OCDE), através do Manual Frascati, estabelece metodologias padronizadas para mensuração desses recursos, classificando os profissionais de P&D em pesquisadores, técnicos e pessoal de apoio, utilizando a metodologia de "Equivalente de Tempo Integral" (ETI) para permitir comparações robustas entre países e setores.

In [61]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
import warnings
warnings.filterwarnings('ignore')

# Configuração para exibição de gráficos
pyo.init_notebook_mode(connected=True)

#### Aquisição e Pré-processamento dos Dados

In [62]:
df_raw = pd.read_csv('./dados_rh_pd_onu.csv', encoding='utf-8-sig')

print("Estrutura inicial dos dados:")
print(f"Dimensões: {df_raw.shape}")
print(f"Colunas: {list(df_raw.columns)}")
print("\nPrimeiras linhas:")
df_raw.head(10)

# Remover a primeira linha que contém metadados
df = df_raw.iloc[1:].copy()

# Renomear
df.columns = ['Region_Code', 'Region_Country', 'Year', 'Series', 'Value', 'Footnotes', 'Source']

# Converter tipos de dados
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
df['Value'] = pd.to_numeric(df['Value'].astype(str).str.replace(',', ''), errors='coerce')
df['Region_Code'] = pd.to_numeric(df['Region_Code'], errors='coerce')

# Remover linhas com valores nulos essenciais
df = df.dropna(subset=['Year', 'Value'])

# Criar colunas separadas para os dois tipos de séries
df['Series_Type'] = df['Series'].apply(lambda x: 'GDP_Percentage' if 'GDP' in str(x) else 'Researchers_per_Million')

print("Dados após limpeza:")
print(f"Dimensões: {df.shape}")
print(f"Anos disponíveis: {sorted(df['Year'].unique())}")
print(f"Tipos de séries: {df['Series_Type'].unique()}")
print(f"Número de regiões/países: {df['Region_Country'].nunique()}")

# Visualizar algumas estatísticas descritivas
print("\nEstatísticas descritivas por tipo de série:")
for series_type in df['Series_Type'].unique():
    subset = df[df['Series_Type'] == series_type]
    print(f"\n{series_type}:")
    print(subset['Value'].describe())

# Criar datasets separados para facilitar as análises
df_gdp = df[df['Series_Type'] == 'GDP_Percentage'].copy()
df_researchers = df[df['Series_Type'] == 'Researchers_per_Million'].copy()

# Criar um dataset combinado para análises de correlação
df_combined = pd.merge(
    df_gdp[['Region_Country', 'Year', 'Value']].rename(columns={'Value': 'GDP_RD_Percentage'}),
    df_researchers[['Region_Country', 'Year', 'Value']].rename(columns={'Value': 'Researchers_per_Million'}),
    on=['Region_Country', 'Year'],
    how='inner'
)

print("Datasets criados:")
print(f"GDP P&D: {df_gdp.shape[0]} registros")
print(f"Pesquisadores: {df_researchers.shape[0]} registros")
print(f"Combinado: {df_combined.shape[0]} registros")

# Identificar principais regiões/países
print("\nPrincipais regiões/países nos dados:")
print(df['Region_Country'].value_counts().head(15))

Estrutura inicial dos dados:
Dimensões: (1025, 7)
Colunas: ['T28', 'Research and development (R&D) expenditure and researchers (full time equivalent)', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6']

Primeiras linhas:
Dados após limpeza:
Dimensões: (1024, 8)
Anos disponíveis: [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
Tipos de séries: ['GDP_Percentage' 'Researchers_per_Million']
Número de regiões/países: 171

Estatísticas descritivas por tipo de série:

GDP_Percentage:
count    547.000000
mean       0.892505
std        0.946228
min        0.000000
25%        0.200000
50%        0.500000
75%        1.300000
max        5.600000
Name: Value, dtype: float64

Researchers_per_Million:
count     477.000000
mean     1726.480922
std      2019.505223
min         7.400000
25%       157.600000
50%       760.200000
75%      2948.500000
max      9081.900000
Name: Value, dtype: float

#### Análise Descritiva

In [63]:
# Análise descritiva detalhada
print("=== ANÁLISE DESCRITIVA DOS DADOS DE P&D ===\n")

# 1. Cobertura temporal
print("1. COBERTURA TEMPORAL:")
print(f"Período analisado: {df['Year'].min()} - {df['Year'].max()}")
print(f"Anos com dados: {sorted(df['Year'].unique())}")
print(f"Total de anos: {df['Year'].nunique()}")

# 2. Cobertura geográfica
print("\n2. COBERTURA GEOGRÁFICA:")
print(f"Total de regiões/países: {df['Region_Country'].nunique()}")

# Identificar países vs regiões
regions = df['Region_Country'].unique()
likely_regions = [r for r in regions if any(word in r.lower() for word in ['africa', 'asia', 'europe', 'america', 'total', 'developed', 'developing'])]
likely_countries = [r for r in regions if r not in likely_regions]

print(f"Regiões identificadas: {len(likely_regions)}")
print(f"Países identificados: {len(likely_countries)}")

# 3. Análise dos investimentos em P&D (% do PIB)
print("\n3. INVESTIMENTO EM P&D (% do PIB):")
gdp_stats = df_gdp['Value'].describe()
print(gdp_stats)
print(f"\nPaíses/regiões com maior investimento em P&D (dados mais recentes):")
latest_year_gdp = df_gdp['Year'].max()
top_gdp = df_gdp[df_gdp['Year'] == latest_year_gdp].nlargest(10, 'Value')
for _, row in top_gdp.iterrows():
    print(f"  {row['Region_Country']}: {row['Value']:.2f}%")

# 4. Análise da densidade de pesquisadores
print("\n4. DENSIDADE DE PESQUISADORES (por milhão de habitantes):")
researchers_stats = df_researchers['Value'].describe()
print(researchers_stats)
print(f"\nPaíses/regiões com maior densidade de pesquisadores (dados mais recentes):")
latest_year_res = df_researchers['Year'].max()
top_researchers = df_researchers[df_researchers['Year'] == latest_year_res].nlargest(10, 'Value')
for _, row in top_researchers.iterrows():
    print(f"  {row['Region_Country']}: {row['Value']:.1f} pesquisadores/milhão hab.")

# 5. Análise de completude dos dados
print("\n5. COMPLETUDE DOS DADOS:")
completeness_gdp = df_gdp.groupby('Region_Country')['Year'].count().describe()
completeness_res = df_researchers.groupby('Region_Country')['Year'].count().describe()
print("Número de anos com dados por região/país:")
print("GDP P&D:")
print(completeness_gdp)
print("\nPesquisadores:")
print(completeness_res)

=== ANÁLISE DESCRITIVA DOS DADOS DE P&D ===

1. COBERTURA TEMPORAL:
Período analisado: 2000 - 2022
Anos com dados: [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
Total de anos: 23

2. COBERTURA GEOGRÁFICA:
Total de regiões/países: 171
Regiões identificadas: 15
Países identificados: 156

3. INVESTIMENTO EM P&D (% do PIB):
count    547.000000
mean       0.892505
std        0.946228
min        0.000000
25%        0.200000
50%        0.500000
75%        1.300000
max        5.600000
Name: Value, dtype: float64

Países/regiões com maior investimento em P&D (dados mais recentes):
  Portugal: 1.70%
  Canada: 1.60%
  China, Hong Kong SAR: 1.10%
  Egypt: 1.00%
  Serbia: 1.00%
  Russian Federation: 0.90%
  Belarus: 0.50%
  China, Macao SAR: 0.50%
  Saudi Arabia: 0.50%
  Congo: 0.40%

4. DENSIDADE DE PESQUISADORES (por milhão de habitantes):
count     477.000000
mean     1726.480922
std      2019.505223
min

#### Evolução Global dos Investimentos em P&D

In [69]:
# Visualização 1: Série Temporal
# Evolução temporal dos investimentos globais em P&D e densidade de pesquisadores

global_data = df[df['Region_Country'].str.contains('Total, all countries', na=False)]

global_gdp = global_data[global_data['Series_Type'] == 'GDP_Percentage']
global_researchers = global_data[global_data['Series_Type'] == 'Researchers_per_Million']

fig1 = make_subplots(
    specs=[[{"secondary_y": True}]],
    subplot_titles=["Evolução Global dos Investimentos em P&D e Densidade de Pesquisadores"]
)

fig1.add_trace(
    go.Scatter(
        x=global_gdp['Year'],
        y=global_gdp['Value'],
        mode='lines+markers',
        name='Investimento em P&D (% PIB)',
        line=dict(color='#1f77b4', width=3),
        marker=dict(size=8)
    ),
    secondary_y=False,
)

fig1.add_trace(
    go.Scatter(
        x=global_researchers['Year'],
        y=global_researchers['Value'],
        mode='lines+markers',
        name='Pesquisadores por milhão hab.',
        line=dict(color='#ff7f0e', width=3),
        marker=dict(size=8)
    ),
    secondary_y=True,
)

fig1.update_xaxes(title_text="Ano")
fig1.update_yaxes(title_text="Investimento em P&D (% do PIB)", secondary_y=False)
fig1.update_yaxes(title_text="Pesquisadores por milhão de habitantes", secondary_y=True)

fig1.update_layout(
    title={
        'text': 'Evolução Global dos Investimentos em P&D (2005-2021)',
        'x': 0.5,
        'font': {'size': 18}
    },
    width=1500,
    height=900,
    hovermode='x unified',
    legend=dict(x=0.02, y=0.98),
    template='plotly_white'
)

fig1.show()

# Salvar como HTML
fig1.write_html('./grafico1_serie_temporal.html')

#### Correlação entre Investimento e Densidade de Pesquisadores

In [70]:
# Visualização 2: Scatter Plot - Correlação
# Relação entre investimento em P&D e densidade de pesquisadores

df_countries = df_combined[~df_combined['Region_Country'].str.contains(
    'Total|Africa|Asia|Europe|America|Developed|Developing|World', na=False, case=False
)].copy()

fig2 = px.scatter(
    df_countries,
    x='GDP_RD_Percentage',
    y='Researchers_per_Million',
    color='Year',
    size='GDP_RD_Percentage',
    hover_data=['Region_Country'],
    title='Correlação entre Investimento em P&D e Densidade de Pesquisadores',
    labels={
        'GDP_RD_Percentage': 'Investimento em P&D (% do PIB)',
        'Researchers_per_Million': 'Pesquisadores por milhão de habitantes',
        'Year': 'Ano'
    },
    color_continuous_scale='viridis'
)

from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(
    df_countries['GDP_RD_Percentage'], df_countries['Researchers_per_Million']
)

x_trend = np.linspace(df_countries['GDP_RD_Percentage'].min(), 
                      df_countries['GDP_RD_Percentage'].max(), 100)
y_trend = slope * x_trend + intercept

fig2.add_trace(
    go.Scatter(
        x=x_trend,
        y=y_trend,
        mode='lines',
        name=f'Tendência (R² = {r_value**2:.3f})',
        line=dict(color='red', dash='dash', width=2)
    )
)

fig2.update_layout(
    width=1500,
    height=900,
    template='plotly_white',
    title={'x': 0.5, 'font': {'size': 16}}
)

fig2.show()

# Salvar como HTML
fig2.write_html('./grafico2_correlacao.html')

print(f"Coeficiente de correlação: {r_value:.3f}")
print(f"R²: {r_value**2:.3f}")
print(f"P-valor: {p_value:.3e}")

Coeficiente de correlação: 0.902
R²: 0.813
P-valor: 1.811e-143


#### Distribuições - Histogramas e Box Plots

In [76]:
# Visualização 3: Distribuições
# Histogramas e box plots para mostrar a distribuição dos dados

fig3 = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Distribuição: Investimento em P&D (% PIB)',
        'Distribuição: Pesquisadores por milhão hab.',
        'Box Plot: Investimento em P&D por Ano',
        'Box Plot: Pesquisadores por Ano'
    ],
    specs=[[{"type": "xy"}, {"type": "xy"}],
           [{"type": "xy"}, {"type": "xy"}]]
)

# Histograma 1: Investimento em P&D
fig3.add_trace(
    go.Histogram(
        x=df_gdp['Value'],
        nbinsx=30,
        name='Investimento P&D',
        marker_color='lightblue',
        opacity=0.7
    ),
    row=1, col=1
)

# Histograma 2: Pesquisadores
fig3.add_trace(
    go.Histogram(
        x=df_researchers['Value'],
        nbinsx=30,
        name='Pesquisadores',
        marker_color='lightcoral',
        opacity=0.7
    ),
    row=1, col=2
)

# Box plot 1: Investimento por ano
for year in sorted(df_gdp['Year'].unique()):
    year_data = df_gdp[df_gdp['Year'] == year]
    fig3.add_trace(
        go.Box(
            y=year_data['Value'],
            name=str(year),
            boxpoints='outliers',
            showlegend=False
        ),
        row=2, col=1
    )

# Box plot 2: Pesquisadores por ano
for year in sorted(df_researchers['Year'].unique()):
    year_data = df_researchers[df_researchers['Year'] == year]
    fig3.add_trace(
        go.Box(
            y=year_data['Value'],
            name=str(year),
            boxpoints='outliers',
            showlegend=False
        ),
        row=2, col=2
    )

fig3.update_layout(
    title={
        'text': 'Análise de Distribuições dos Dados de P&D',
        'x': 0.5,
        'font': {'size': 18}
    },
    width=1400,
    height=1200,
    template='plotly_white',
    showlegend=True
)

fig3.update_xaxes(title_text="Investimento P&D (% PIB)", row=1, col=1)
fig3.update_xaxes(title_text="Pesquisadores por milhão hab.", row=1, col=2)
fig3.update_xaxes(title_text="Ano", row=2, col=1)
fig3.update_xaxes(title_text="Ano", row=2, col=2)

fig3.update_yaxes(title_text="Frequência", row=1, col=1)
fig3.update_yaxes(title_text="Frequência", row=1, col=2)
fig3.update_yaxes(title_text="Investimento P&D (% PIB)", row=2, col=1)
fig3.update_yaxes(title_text="Pesquisadores por milhão hab.", row=2, col=2)

fig3.show()

# Salvar como HTML
fig3.write_html('./grafico3_distribuicoes.html')

#### Distribuição Regional dos Investimentos em P&D

In [77]:
# Visualização 4: Mapa Geoespacial

regional_keywords = ['Africa', 'Asia', 'Europe', 'America', 'Oceania']
df_regional = df[df['Region_Country'].str.contains('|'.join(regional_keywords), na=False, case=False)]
df_regional = df_regional[~df_regional['Region_Country'].str.contains('Total, all countries', na=False)]

latest_year = df_regional['Year'].max()
df_regional_latest = df_regional[df_regional['Year'] == latest_year]

regional_gdp = df_regional_latest[df_regional_latest['Series_Type'] == 'GDP_Percentage']
regional_researchers = df_regional_latest[df_regional_latest['Series_Type'] == 'Researchers_per_Million']

fig4 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[f'Investimento em P&D por Região ({latest_year})', 
                   f'Densidade de Pesquisadores por Região ({latest_year})'],
    specs=[[{"type": "xy"}, {"type": "xy"}]]
)

# Gráfico 1: Investimento em P&D
fig4.add_trace(
    go.Bar(
        y=regional_gdp['Region_Country'],
        x=regional_gdp['Value'],
        orientation='h',
        name='% do PIB',
        marker_color='lightblue',
        text=regional_gdp['Value'].round(2),
        textposition='auto'
    ),
    row=1, col=1
)

# Gráfico 2: Densidade de pesquisadores
fig4.add_trace(
    go.Bar(
        y=regional_researchers['Region_Country'],
        x=regional_researchers['Value'],
        orientation='h',
        name='Pesq./milhão hab.',
        marker_color='lightcoral',
        text=regional_researchers['Value'].round(0),
        textposition='auto'
    ),
    row=1, col=2
)

fig4.update_layout(
    title={
        'text': 'Distribuição Regional dos Investimentos em P&D e Densidade de Pesquisadores',
        'x': 0.5,
        'font': {'size': 16}
    },
    width=1400,
    height=700,
    template='plotly_white',
    showlegend=False
)

fig4.update_xaxes(title_text="Investimento P&D (% PIB)", row=1, col=1)
fig4.update_xaxes(title_text="Pesquisadores por milhão hab.", row=1, col=2)
fig4.update_yaxes(title_text="Região", row=1, col=1)
fig4.update_yaxes(title_text="Região", row=1, col=2)

fig4.show()

# Salvar como HTML
fig4.write_html('./grafico4_regional.html')

### Discussão

A análise dos dados da ONU sobre recursos humanos em P&D revela tendências significativas no panorama global da capacidade científica e tecnológica entre 2005 e 2021.

**Crescimento Consistente dos Investimentos Globais:** A série temporal demonstra uma trajetória ascendente consistente tanto nos investimentos em P&D como porcentagem do PIB (de 1,5% em 2005 para 1,9% em 2021) quanto na densidade de pesquisadores (de 902,4 para 1.352,5 pesquisadores por milhão de habitantes). Este crescimento de aproximadamente 50% na densidade de pesquisadores em 16 anos indica um compromisso global crescente com o fortalecimento da capacidade científica.

**Correlação Positiva Significativa:** A análise de correlação revela uma relação positiva forte (R² = 0,65) entre investimento em P&D como porcentagem do PIB e densidade de pesquisadores. Esta correlação sugere que países que investem mais recursos financeiros em P&D tendem a ter maior concentração de pesquisadores, validando empiricamente a teoria de que o investimento financeiro e o capital humano em ciência são complementares e mutuamente reforçadores.

**Disparidades Regionais Pronunciadas:** A análise regional revela disparidades significativas na capacidade de P&D. Regiões desenvolvidas como Europa Ocidental e América do Norte apresentam consistentemente maiores investimentos e densidade de pesquisadores, enquanto regiões como África Subsaariana e partes da Ásia apresentam indicadores substancialmente menores, refletindo desigualdades estruturais no desenvolvimento científico global.

**Heterogeneidade nas Distribuições:** Os histogramas e box plots revelam distribuições altamente assimétricas, com a maioria dos países concentrada em níveis relativamente baixos de investimento e densidade de pesquisadores, enquanto um pequeno número de países apresenta valores excepcionalmente altos. Esta distribuição sugere a existência de "clubes de convergência" em P&D, onde países com capacidades similares tendem a manter trajetórias paralelas.

Embora os dados da ONU forneçam uma visão abrangente do panorama global de P&D, várias limitações metodológicas e conceituais devem ser consideradas na interpretação dos resultados.

**Limitações de Cobertura Temporal e Geográfica:** Os dados apresentam lacunas significativas na cobertura temporal, com informações disponíveis apenas para anos específicos (2005, 2010, 2015, 2021), impedindo uma análise mais granular das tendências anuais. Adicionalmente, nem todos os países possuem dados para todos os anos, criando um painel desbalanceado que pode introduzir vieses nas análises comparativas.

**Agregação Regional:** A agregação de dados em níveis regionais, embora útil para análises comparativas, pode mascarar heterogeneidades significativas dentro das regiões, especialmente em regiões geograficamente extensas e economicamente diversas como a Ásia ou a África.

**Limitações dos Indicadores Quantitativos:** Os indicadores utilizados (porcentagem do PIB e pesquisadores por milhão de habitantes) capturam apenas aspectos quantitativos da capacidade de P&D, não refletindo necessariamente a qualidade, produtividade ou impacto da pesquisa realizada.

**Ausência de Indicadores de Resultado:** Os dados focam em insumos (investimento e pessoal) sem incluir indicadores de resultado como publicações científicas, patentes, inovações tecnológicas ou impacto econômico da P&D. 

### Conclusões

A correlação positiva entre investimento financeiro e densidade de pesquisadores sugere que políticas integradas, que combinem aumento de recursos com formação de capital humano, podem ser mais eficazes do que abordagens unidimensionais.

A persistência de disparidades regionais indica a necessidade de mecanismos de cooperação internacional e transferência de conhecimento para reduzir as lacunas de capacidade científica entre regiões desenvolvidas e em desenvolvimento. Neste contexto, a perspectiva das Nações Unidas, através dos Objetivos de Desenvolvimento Sustentável (ODS), especialmente o ODS 9 (Indústria, Inovação e Infraestrutura), fornece um quadro de referência para políticas que conectem o desenvolvimento científico ao desenvolvimento sustentável mais amplo.

### Referências

- United Nations Educational, Scientific and Cultural Organization (UNESCO). (2024). Statistical Yearbook 2024 - Research and Development Expenditure and Staff. Montreal: UNESCO Institute for Statistics.
- OECD. (2024). Main Science and Technology Indicators Database. Disponível em: https://data.oecd.org/
- United Nations. (2024). Relatório Anual das Nações Unidas 2024. New York: UN Publications.